<a href="https://colab.research.google.com/github/tcapelle/hackercup_rag/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{rag-hackercup} -->

<img src="http://wandb.me/logo-im-png" width="400" alt="Weights & Biases" />


# Introduction


</a>


In this notebook, we will build a few Code Generation agents for the [HackerCup AI](https://hackercupai.github.io/) challenge.

We will build three different agents using different techniques and evaluate them using [W&B Weave](https://weave-docs.wandb.ai/).


<img src="https://raw.githubusercontent.com/wandb/weave/master/docs/static/img/evals-hero.png" width="800" height="450">

A more detailed walkthough of the approach we will use in this notebook can be found in the following Youtube video:
Hint: Click on the image to watch the video 😎

<a target="_blank" href="https://www.youtube.com/watch?v=cObBj2UpWK8">
<img src="https://img.youtube.com/vi/cObBj2UpWK8/0.jpg" width="600" height="450">
</a>

## Weave


Weave is a lightweight toolkit for tracking and evaluating LLM applications, built by Weights & Biases. We will use the following weave to trace and evaluate the various agents we build.

We will use Weave to keep track and evaluate the different agents we build.

Our goal is to bring rigor, best-practices, and composability to the inherently experimental process of developing AI applications, without introducing cognitive overhead.

If you want to learn more about Weave, you can [get started](https://weave-docs.wandb.ai/quickstart) by decorating Python functions with `@weave.op`.

## Setup 

**Note: You need to run this cell only once**
We will clone the starter-kits repo
Set the rag folder as our working directory
and install the dependencies for the project.

**You can comment out the cell after you have run it once.**

In [1]:
# Clone the starter-kits repo
!git clone https://github.com/tcapelle/hackercup_rag
# Change directory to the rag folder. Running the next line twice in the same session will raise an error.
%cd hackercup_rag
# Install dependencies
!pip install -r requirements.txt

fatal: destination path 'hackercup_rag' already exists and is not an empty directory.
/Users/tcapelle/work/hackercup_rag/hackercup_rag


In [ ]:
import weave

WEAVE_PROJECT = "hackercup"
weave_client = weave.init(WEAVE_PROJECT)

Logged in as Weights & Biases user: capecape.
View Weave data at https://wandb.ai/capecape/hackercup/weave


## Dataset
We will use [HackerCup dataset](https://huggingface.co/datasets/hackercupai/hackercup) in this notebook.

Specifically, the **practice** dataset from the **2023** season.

We have already processed the dataset and saved it as a [`weave.Dataset`](https://weave-docs.wandb.ai/guides/core-types/datasets/). You can either use the Dataset by running the next cell or download the dataset using the instructions below.

We will use the dataset to load some practice problems and solutions from the HackerCup dataset and evaluate our agents on it.

In [ ]:
import os

os.environ["BASE_URL"] = "http://195.242.24.252:8000/v1"
os.environ["MAX_TOKENS"] = "2048"
os.environ["WEAVE_PARALLELISM"] = 1

from utils import (
    Problem,
    FAST_LLM,
    STRONG_LLM,
)


practice_dataset_uri = "weave:///parambharat/hackercup/object/practice_dataset:R35fXf9N3FE2IOesg7bRPaPAxiE9YbpirhXO9HcHs8w"
problems_dataset = weave.ref(practice_dataset_uri).get().rows[:]
problems = list(map(lambda x: Problem(**x), problems_dataset))
problem = problems[0]
print("Sample Problem:\n\n", problem.model_dump_json(indent=2))

Sample Problem:

 {
  "problem_dir": "data/2023/practice",
  "problem_name": "two_apples_a_day",
  "problem_description": "“An apple a day keeps the doctor away” is Steve’s motto. His other motto, “You can never have too much of a good thing,” holds true for both apples and mottos. Steve would like to eat two apples per day for the next \\(N\\) days, but with strict adherence to his third motto “Consistency is key.” Specifically, he’d like the sum of the two apple weights he eats over the next \\(N\\) days to be the same for each day.\n\nSteve has already purchased \\(2*N-1\\) apples, the \\(i\\)th of which weighs \\(A_i\\) ounces. He'd like to buy one more apple that's as light as possible to fulfill his goal. Steve can buy an apple of any positive integer weight in ounces from the store. Is it possible for him to reach his goal, and if so, what weight apple should he buy?\n\n{{PHOTO_ID:1563872647765708|WIDTH:600}}\n\n\n*The above image depicts the solution to the first sample. Each d

Alternatively, you can download the dataset by running the download script from the [submit-first-solution](https://github.com/HackerCupAI/starter-kits/tree/main/submit_first_solution). Specifically, you can run the following command to download the dataset:

```bash
python download.py --year 2023 --dataset_folder data
```


This should create a `dataset` folder with the problems and solutions.

Here's an example of what the data looks like for the `dim_sum_delivery` problem from the `2023` season:

```
data/dataset/2023/practice
...
├── dim_sum_delivery.cpp
├── dim_sum_delivery.in
├── dim_sum_delivery.md
├── dim_sum_delivery.out
├── dim_sum_delivery_sample_input.txt
├── dim_sum_delivery_sample_output.txt
├── dim_sum_delivery_sol.md
...
```

Each problem has a `in`, `out`, `md`, `cpp`, and `sol` file.

The `in` file contains the input data for the problem.
The `out` file contains the expected output for the problem.
The `md` file contains the problem statement.
The `cpp` file contains the source code to the solution.
The `sol` file contains the detailed solution to the problem.
The `sample_input.txt` and `sample_output.txt` files contain the sample input and output for the problem. These are the test cases that will be available to the agent during development and evaluation.

In [ ]:
import asyncio
import logging

from nest_asyncio import apply

apply()

# Some logging to see the progress
logging.basicConfig(
    format="%(asctime)s : %(levelname)s : %(message)s", level=logging.INFO
)

logger = logging.getLogger(__name__)

Now that we have a way to load a problem and evaluate a solution, let's define a prompt to solve the problem and create a simple agent to solve the problem. 

Here'e one such prompt we will use to solve the problem, it contains instructions for the model on how to solve the problem and the format of the response we expect from the model. Feel free to tweak the prompt if you like but this should work decently well for our use case.

## Reflection Agent



While the RAG agent is an improvement over the zero-shot agent, it's still not perfect.
It's still susceptible to hallucinations and incorrect solutions. 
One way to mitigate this is to use reflection.
We can use another LLM call to reflect on the solution and test results and improve it.
We can then use the improved solution to generate new few-shot examples and repeat the process in a loop until we converge to a solution or the iteration limit is reached.

Again, this is not the best approach to solve the problem and has a lot of room for improvement, but it should help us get towards a working solution.

Here are the reflection instructions we will provide to the LLM to reflect on the solution and test results, feel free to change the instructions to improve the agent's performance.

In [ ]:
from agent import rag_solver
from retriever import Retriever

retriever = Retriever()

from agent import REFLECTION_INSTRUCTIONS, rework_solution

print(REFLECTION_INSTRUCTIONS)

2024-09-05 22:56:39,984 : INFO : PyTorch version 2.4.0 available.
2024-09-05 22:56:42,080 : INFO : Use pytorch device_name: mps
2024-09-05 22:56:42,080 : INFO : Load pretrained SentenceTransformer: jinaai/jina-embeddings-v2-base-code
2024-09-05 22:56:49,314 : DEBUG : Building index from IDs objects      
                                                                               

You are a world-class competitive programmer with a keen eye for detail and problem solving. 
Your expertise is in algorithms and data structures. 
You have incorrectly answered the following programming problem. 
Your task is to reflect on the problem, your solution, and the correct answer.
You will then use this information help you answer the same question in the future. 
First, explain why you answered the question incorrectly.
Second, list the keywords that describe the type of your errors from most general to most specific.
Third, solve the problem again, step-by-step, based on your knowledge of the correct answer.
Fourth, create a list of detailed instructions to help you correctly solve this problem in the future.
Finally, create a list of general advice to help you solve similar types of problems in the future.
Be concise in your response; however, capture all of the essential information.

{problem}
<incorrect_solution>
{incorrect_solution}
</incorrect_solution>
<test_report>

In [ ]:
@weave.op
async def rag_solver_with_reflection(
        retriever: Retriever,
        problem: Problem,
        model: str = FAST_LLM,
        temperature: float = 0.1,
        max_iterations: int = 2,
        timeout: int = 10,
):
    num_iterations = 0
    test_report = "failed"
    solution = None
    while not test_report == "passed" and num_iterations < max_iterations:
        rag_result = await rag_solver(
            retriever=retriever,
            problem=problem,
            timeout=timeout,
            model=model,
            temperature=temperature,
        )
        solution = rag_result["solution"]
        test_report = rag_result["test_report"]
        if test_report == "passed":
            return rag_result
        rework_result = await rework_solution(
            problem=problem,
            incorrect_solution=solution,
            test_report=test_report,
            model=model,
            temperature=temperature,
            timeout=timeout,
        )
        solution = rework_result["solution"]
        test_report = rework_result["test_report"]
        if test_report == "passed":
            return {
                "solution": solution,
                "stage": "reflection",
                "test_report": test_report,
            }
        num_iterations += 1
    logger.info("Failed to generate a solution")
    return {"solution": solution, "stage": "failed", "test_report": test_report}

In [ ]:
reflection_result = await rag_solver_with_reflection(
    retriever, problem, max_iterations=2, timeout=30
)

print("*" * 80)
print(reflection_result["solution"].source_code)
print("*" * 80)
print(reflection_result["test_report"])

2024-09-05 22:56:52,762 : INFO : Drafting intial zero-shot solution
22:57:10 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 22:57:10,333 : INFO : 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 22:57:16,959 : INFO : HTTP Request: POST http://195.242.24.252:8000/v1/chat/completions "HTTP/1.1 200 OK"
22:57:17 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 22:57:17,598 : INFO : 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 22:57:22,799 : INFO : HTTP Request: POST http://195.242.24.252:8000/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-05 22:57:25,034 : INFO : Draft solution result: "WRONG ANSWER!!\n\n<expected>\n'Case #1: 4\nCase #2: 7\nCase #3: 1\nCase #4: -1\nCase #5: 6\nCase #6: -1\nCase #7: 1000000002\n'\n</expected>\n---\n<got>\n'Case #1: 1\nCase #2: 3\nCase #3: 1\nCase #4: 2\nCa


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



23:02:03 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 23:02:03,997 : INFO : 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 23:02:04,056 : INFO : Retrying request to /chat/completions in 0.797548 seconds
23:02:04 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 23:02:04,154 : INFO : 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral



Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



2024-09-05 23:02:04,214 : INFO : Retrying request to /chat/completions in 0.783185 seconds
2024-09-05 23:02:04,673 : INFO : Retrying request to /chat/completions in 1.674434 seconds
2024-09-05 23:02:04,760 : INFO : Retrying request to /chat/completions in 1.651550 seconds
2024-09-05 23:02:04,864 : INFO : Retrying request to /chat/completions in 1.774962 seconds
2024-09-05 23:02:04,910 : INFO : Retrying request to /chat/completions in 1.911112 seconds
2024-09-05 23:02:05,052 : INFO : Retrying request to /chat/completions in 1.948356 seconds
23:02:06 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 23:02:06,426 : INFO : 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
23:02:06 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 23:02:06,490 : INFO : Retrying request to /chat/completions in 0.894631 seconds
2024-09-05 23:02:06,489 : INFO :

...


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



2024-09-05 23:02:07,456 : INFO : Retrying request to /chat/completions in 1.806692 seconds
2024-09-05 23:02:07,509 : INFO : Retrying request to /chat/completions in 1.858282 seconds
2024-09-05 23:02:07,727 : INFO : Retrying request to /chat/completions in 1.787996 seconds
2024-09-05 23:02:07,978 : INFO : Retrying request to /chat/completions in 1.731372 seconds
2024-09-05 23:02:07,980 : INFO : Retrying request to /chat/completions in 1.618331 seconds
2024-09-05 23:02:17,239 : INFO : HTTP Request: POST http://195.242.24.252:8000/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-05 23:02:18,603 : INFO : HTTP Request: POST http://195.242.24.252:8000/v1/chat/completions "HTTP/1.1 200 OK"
2024-09-05 23:02:22,523 : INFO : HTTP Request: POST http://195.242.24.252:8000/v1/chat/completions "HTTP/1.1 200 OK"
23:02:22 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 23:02:22,532 : INFO : 
LiteLLM completion() model= open-mistral-nemo-2


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



InstructorRetryException: RetryError[<Future at 0x3388790d0 state=finished raised APIError>]

2024-09-05 23:02:42,624 : INFO : HTTP Request: POST http://195.242.24.252:8000/v1/chat/completions "HTTP/1.1 500 Internal Server Error"
2024-09-05 23:02:42,627 : INFO : Retrying request to /chat/completions in 0.945439 seconds
2024-09-05 23:02:58,686 : INFO : HTTP Request: POST http://195.242.24.252:8000/v1/chat/completions "HTTP/1.1 200 OK"


Great, now, we are ready to evaluate a more complex agent that uses reflection
This agent will try to solve the problem using the retriever
and if it fails, it will ask the model to reflect on the problem
and then re-work the solution
and repeat this process for a fixed number of iterations
or until the solution is correct or the iteration limit is reached

But the best part is that we can use the same evaluation framework we used for the zero-shot and RAG agent to evaluate the RAG reflection agent.

In [ ]:
class RAGReflectionAgent(weave.Model):
    retriever: Retriever
    max_iterations: int = 2
    timeout: int = 30
    model: str = STRONG_LLM
    temperature: float = 0.0

    @weave.op
    async def predict(self, problem: Problem):
        return await rag_solver_with_reflection(
            self.retriever,
            Problem(**problem),
            model=self.model,
            temperature=self.temperature,
            max_iterations=self.max_iterations,
            timeout=self.timeout,
        )

In [ ]:
# This is a simple depection of the evaluation.
# We expect the output to be `"passed"` for all the problems if the agent is working correctly.
examples = [{"problem": problem, "expected": "passed"} for problem in problems]


# A simple scorer that checks if the code generated by agent passed the test case
@weave.op
def scorer(expected: str, model_output: dict) -> dict:
    return {"passed": expected == model_output["test_report"]}


# This is a simple evaluation that checks if the code generated by agent passed the test
evaluation = weave.Evaluation(dataset=examples, scorers=[scorer])

In [ ]:
# Evaluate the RAG reflection agent for all the models and temperatures
tasks = []

LLM = STRONG_LLM
eval_temperatures = [0.0, 0.5]


for temperature in eval_temperatures:
        rag_reflection_agent = RAGReflectionAgent(
            retriever=retriever, model=LLM, temperature=temperature, timeout=30
        )
        rag_reflection_results = evaluation.evaluate(rag_reflection_agent)
        tasks.append(rag_reflection_results)
rag_reflection_results = await asyncio.gather(*tasks)

2024-09-05 23:04:42,922 : INFO : Drafting intial zero-shot solution
23:04:42 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= mistral-large-latest; provider = mistral
2024-09-05 23:04:42,929 : INFO : 
LiteLLM completion() model= mistral-large-latest; provider = mistral
2024-09-05 23:04:43,248 : INFO : Drafting intial zero-shot solution
23:04:43 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= mistral-large-latest; provider = mistral
2024-09-05 23:04:43,255 : INFO : 
LiteLLM completion() model= mistral-large-latest; provider = mistral
2024-09-05 23:04:43,575 : INFO : Drafting intial zero-shot solution
23:04:43 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= mistral-large-latest; provider = mistral
2024-09-05 23:04:43,582 : INFO : 
LiteLLM completion() model= mistral-large-latest; provider = mistral
2024-09-05 23:04:43,923 : INFO : Drafting intial zero-shot solution
23:04:43 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= mistral-large-la

Evaluated 1 of 5 examples

2024-09-05 23:05:27,607 : INFO : HTTP Request: POST http://195.242.24.252:8000/v1/chat/completions "HTTP/1.1 200 OK"
23:05:27 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 23:05:27,615 : INFO : 
LiteLLM completion() model= open-mistral-nemo-2407; provider = mistral
2024-09-05 23:05:27,675 : INFO : HTTP Request: POST http://195.242.24.252:8000/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-09-05 23:05:27,676 : INFO : Retrying request to /chat/completions in 0.880277 seconds
2024-09-05 23:05:28,330 : INFO : HTTP Request: POST http://195.242.24.252:8000/v1/chat/completions "HTTP/1.1 500 Internal Server Error"
2024-09-05 23:05:28,331 : INFO : Retrying request to /chat/completions in 1.691389 seconds
2024-09-05 23:05:28,381 : INFO : HTTP Request: POST http://195.242.24.252:8000/v1/chat/completions "HTTP/1.1 500 Internal Server Error"
2024-09-05 23:05:28,383 : INFO : Retrying request to /chat/completions in 1.936


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.


Give Feedback / Get Help: https://github.com/BerriAI/litellm/issues/new
LiteLLM.Info: If you need to debug this error, use `litellm.set_verbose=True'.



2024-09-05 23:05:30,285 : INFO : HTTP Request: POST http://195.242.24.252:8000/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
2024-09-05 23:05:30,291 : INFO : Retrying request to /chat/completions in 0.987926 seconds
2024-09-05 23:05:30,369 : INFO : HTTP Request: POST http://195.242.24.252:8000/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
23:05:30 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= mistral-large-latest; provider = mistral
2024-09-05 23:05:30,376 : INFO : 
LiteLLM completion() model= mistral-large-latest; provider = mistral
2024-09-05 23:05:30,381 : INFO : HTTP Request: POST http://195.242.24.252:8000/v1/chat/completions "HTTP/1.1 429 Too Many Requests"
23:05:30 - LiteLLM:INFO: utils.py:2975 - 
LiteLLM completion() model= mistral-large-latest; provider = mistral
2024-09-05 23:05:30,389 : INFO : 
LiteLLM completion() model= mistral-large-latest; provider = mistral
2024-09-05 23:05:30,432 : INFO : HTTP Request: POST http://195.242.24.252:8000/v1/cha

...

Traceback (most recent call last):
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/litellm/llms/openai.py", line 1089, in acompletion
    headers, response = await self.make_openai_chat_completion_request(
                        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/litellm/llms/openai.py", line 793, in make_openai_chat_completion_request
    raise e
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/litellm/llms/openai.py", line 781, in make_openai_chat_completion_request
    await openai_aclient.chat.completions.with_raw_response.create(
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/openai/_legacy_response.py", line 367, in wrapped
    return cast(LegacyAPIResponse[R], await func(*args, **kwargs))
                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.1

Predict and score failed

Traceback (most recent call last):
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/weave/flow/eval.py", line 287, in eval_example
    eval_row = await self.predict_and_score(model, example)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/weave/trace/op.py", line 333, in wrapper
    res, _ = await _execute_call(wrapper, call, *args, **kwargs)  # type: ignore
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/weave/trace/op.py", line 213, in _call_async
    return handle_exception(e)
           ^^^^^^^^^^^^^^^^^^^
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/weave/trace/op.py", line 211, in _call_async
    res = await func(*args, **kwargs)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tcapelle/miniforge3/envs/weave/lib/python3.11/site-packages/weave/fl

Evaluated 5 of 5 examples

Evaluation summary
{'scorer': None}

Okay, that completes the demo!

Key takeaways from this demo:
1. We tried to solve some challenging competitive programming problems using LLM agents.
2. We tried three different agents:
    - Zero-shot agent
    - RAG agent
    - RAG reflection agent
3. We used Weave to evaluate the agents and compare their performance.

We hope you found this demo useful and interesting and that it gave you some ideas on how to use LLM agents to solve challenging problems.
